In [36]:
import tushare as ts
import os
import datetime
import pymongo
import pandas
import copy
import openpyxl
import json

def ConnectDB():
    client = pymongo.MongoClient("127.0.0.1", 27017)
    return client

def Today():
    today = datetime.datetime.now().replace(hour=0, minute=0, second=0, microsecond=0)
    return today

def SaveResult(client, data):
    db = client.Result
    collection = db.Recommend
    collection.replace_one({
        "code": data["code"],
        "date": data["date"],
        "strategy": data["strategy"]
    }, data, upsert=True)
    
def OpenXlsx(sheet_name):
    file_name = Today().strftime("%Y%m%d.xlsx")
    path = os.path.expanduser("~/Desktop/" + file_name)

    if os.path.exists(path):
        wb = openpyxl.load_workbook(path)
    else:
        wb = openpyxl.Workbook()
        del wb["Sheet"]
    if sheet_name in wb.sheetnames:
        del wb[sheet_name]
    ws = wb.create_sheet(sheet_name)
    return wb, ws, path

def WriteHead(df, ws, offset=None):
    head_alignment = openpyxl.styles.Alignment(horizontal="center", vertical="center")
    head_fill = openpyxl.styles.fills.PatternFill(
        fgColor="D7E4BC",
        fill_type="solid",
        patternType="solid")
    head_font = openpyxl.styles.fonts.Font(
        bold=True,
        sz=14
    )
    if not offset:
        rowStart = 1
        columnStart = 1
    else:
        rowStart = offset[0]
        columnStart = offset[1]
        
    for i, v in enumerate(df.columns.values):
        cell = ws.cell(row=rowStart, column=i+columnStart)
        cell.value = v
        cell.alignment = head_alignment
        cell.fill = head_fill
        cell.font = head_font

In [2]:
def Strategy_1():
    client = ConnectDB()
    
    startDate = datetime.datetime.now() + datetime.timedelta(days=-60)
    stocks = client.Common.Stock.find({"foundTime": {"$lt": startDate}})
    threshold = 9.9
    today = Today()
    found = []
    minDays = 1000

    for item in stocks:
        code = item["code"]
        collection = client.Trade[code]
        res = list(collection.find({}, limit=4, sort=[("date", pymongo.DESCENDING)]))
        if len(res) < 4:
            continue
        changes = []
        for i in range(3):
            if res[i+1]["close"] == 0:
                d = 0
            else:
                d = (res[i]["close"] / res[i+1]["close"] - 1) * 100
            changes.append(d)
        if changes[0] < threshold or (changes[1] <= threshold and changes[2] <= threshold):
            continue
        days = (today - res[3]["date"]).days
        if days > minDays:
            continue
        if days < minDays:
            minDays = days
            found = []
        data = {
            "code": code,
            "name": item["name"],
            "strategy": 1,
            "change_0": changes[0],
            "change_-1": changes[1],
            "change_-2": changes[2],
            
        }
        found.append(data)
        data["date"] = today
        SaveResult(client, data)

    client.close()
    
    if len(found) > 0:
        df = pandas.DataFrame(found, columns=["code", "name", "change_0", "change_-1", "change_-2"])
    else:
        return None
    
    # Write Excel
    sheet_name = u"策略1_强势股"
    wb, ws, path = OpenXlsx(sheet_name)

    ws.row_dimensions[1].height = 20
    for i, v in enumerate([14, 14, 18, 18, 18]):
        ws.column_dimensions[openpyxl.utils.get_column_letter(i+1)].width = v

    head_alignment = openpyxl.styles.Alignment(horizontal="center", vertical="center")
    head_fill = openpyxl.styles.fills.PatternFill(
        fgColor="D7E4BC",
        fill_type="solid",
        patternType="solid")
    head_font = openpyxl.styles.fonts.Font(
        bold=True,
        sz=14
    )
    for i, v in enumerate(df.columns.values):
        cell = ws.cell(row=1, column=i+1)
        cell.value = v
        cell.alignment = head_alignment
        cell.fill = head_fill
        cell.font = head_font

    for i, row in df.iterrows():
        for j, v in enumerate(row.values):
            cell = ws.cell(row=i+2, column=j+1)
            cell.value = v
            if j >= 2:
                cell.number_format = "0.00"

    wb.save(path)
        
    return df

In [3]:
def Strategy_2():
    client = ConnectDB()
    
    today = Today()
    conceptDict = {}
    Threshold = 9.9
    
    for item in client.Common.Stock.find({}):
        code = item["code"]
        collection = client.Trade[code]
        res = collection.find_one({"date": today})
        if not res:
            continue
        if "last_close" not in res:
            continue
        change = (res["close"] / res["last_close"] - 1) * 100
        if change < Threshold:
            continue
    
        for concept in item["concept"]:
            if concept == u"次新股":
                continue
            if concept not in conceptDict:
                conceptDict[concept] = []
            data = {
                "code": item["code"],
                "name": item["name"],
                "change": change
            }
            conceptDict[concept].append(data)
    
    concepts = []
    for k, v in conceptDict.iteritems():
        if len(v) < 3:
            continue
        data = {
            "concept": k,
            "stocks": v,
            "count": len(v),
            "codes": []
        }
        for s in v:
            data["codes"].append(s["code"])
        concepts.append(data)
    
    found = sorted(concepts, key=lambda item: item["count"], reverse=True)
    
    stockDict = {}
    for k in found:
        for s in k["stocks"]:
            code = s["code"]
            if code not in stockDict:
                stockDict[code] = {
                    "code": code,
                    "name": s["name"],
                    "change": s["change"],
                    "concepts": [],
                    "date": today,
                    "strategy": 2,
                }
            stockDict[code]["concepts"].append(k["concept"])
    for code, v in stockDict.iteritems():
        SaveResult(client, v)
        
    client.close()
    
    df = pandas.DataFrame(found, columns=["concept", "count", "codes"])
    
    # Write Excel
    sheet_name = u"策略2_板块启动"
    wb, ws, path = OpenXlsx(sheet_name)

    ws.row_dimensions[1].height = 20
    for i, v in enumerate([14, 14]):
        ws.column_dimensions[openpyxl.utils.get_column_letter(i+1)].width = v

    WriteHead(df, ws)

    for i, row in df.iterrows():
        for j, v in enumerate(row.values):
            if j < 2:
                cell = ws.cell(row=i+2, column=j+1)
                cell.value = v
            else:
                for k, code in enumerate(v):
                    cell = ws.cell(row=i+2, column=j+1+k)
                    cell.value = code

    wb.save(path)
    
    return df

In [4]:
def Strategy_3():
    client = ConnectDB()
    
    stocks = client.Common.Stock.find({})
    today = Today()
    found = []
    minDays = 1000

    for item in stocks:
        code = item["code"]
        collection = client.Trade[code]
        res = list(collection.find({"close": {"$ne": 0}}, limit=10, sort=[("date", pymongo.DESCENDING)]))
        if len(res) < 10:
            continue
        days = (today - res[-1]["date"]).days
        if days > minDays:
            continue
        if res[0]["close"] < res[1]["close"]:
            continue
        if days < minDays:
            minDays = days
            found = []
        v10 = 0
        volumeDays = 2
        vv = 0
        p5 = 0
        for i in range(10):
            v10 = v10 + res[i]["volume"]
            if i < volumeDays:
                vv = vv + res[i]["volume"]
            if i < 5:
                p5 = p5 + res[i]["close"]
        v10 = int(v10 / 10)
        vv = int(vv / volumeDays)
        p5 = p5 / 5
        if v10 == 0:
            continue
        close = res[0]["close"]
        if vv >= v10 * 2 and close >= p5:
            item = {
                "code": code,
                "name": item["name"],
                "volume_" + str(volumeDays): vv,
                "volume_10": v10,
                "price_today": close,
                "price_5": p5,
                "factor": (1.0 * vv / v10)
            }
            found.append(item)

        found = sorted(found, key=lambda item: item["factor"], reverse=True)

    found = found[:10]
    for item in found:
        data = copy.copy(item)
        data["date"] = today
        data["strategy"] = 3
        SaveResult(client, data)
        
    client.close()
    
    if len(found) > 0:
        df = pandas.DataFrame(found,
                              columns=["code", "name", "factor", "price_today",
                                       "price_5", "volume_2", "volume_10"])
    else:
        return None
    
    # Write Excel
    sheet_name = u"策略3_个股启动"
    wb, ws, path = OpenXlsx(sheet_name)

    ws.row_dimensions[1].height = 20
    for i, v in enumerate([14, 14, 14, 14, 14, 14, 14]):
        ws.column_dimensions[openpyxl.utils.get_column_letter(i+1)].width = v

    WriteHead(df, ws)

    for i, row in df.iterrows():
        for j, v in enumerate(row.values):
            cell = ws.cell(row=i+2, column=j+1)
            cell.value = v

    wb.save(path)

    return df

In [5]:
def Strategy_4():
    client = ConnectDB()
    
    today = Today()
    stocks = client.Common.Stock.find({
        "$and": [
            {"value": {"$exists": True}},
            {"value": {"$ne": 0}}
        ]})

    found = []

    for item in stocks:
        code = item["code"]
        collection = client.Trade[code]
        res = collection.find_one({"date": today})
        if not res:
            continue
        if "last_close" not in res:
            continue
        if res["close"] <= res["last_close"]:
            continue
        value = res["tradeValue"] * 10000
        if "amount" in res:
            amount = res["amount"]
        else:
            amount = res["volume"] * (res["open"] + res["close"]) / 2 * 100
        item = {
            "code": code,
            "name": item["name"],
            "value": value / 1e8,
            "amount": amount / 1e8,
            "factor": amount / value
        }
        found.append(item)
        
    found = sorted(found, key=lambda item: item["factor"], reverse=True)[:10]
    for item in found:
        data = copy.copy(item)
        data["date"] = today
        data["strategy"] = 4
        SaveResult(client, data)
        
    client.close()
    
    if len(found) == 0:
        return None
    
    df = pandas.DataFrame(found, columns=["code", "name", "factor", "value", "amount"])
    
    # Write Excel
    sheet_name = u"策略4_换手率"
    wb, ws, path = OpenXlsx(sheet_name)

    ws.row_dimensions[1].height = 20
    for i, v in enumerate([14, 14, 14, 14, 14]):
        ws.column_dimensions[openpyxl.utils.get_column_letter(i+1)].width = v

    WriteHead(df, ws)

    for i, row in df.iterrows():
        for j, v in enumerate(row.values):
            cell = ws.cell(row=i+2, column=j+1)
            cell.value = v

    wb.save(path)
    
    return df

In [10]:
def Strategy_5():
    client = ConnectDB()
    
    stocks = client.Common.Stock.find({})
    today = Today()
    found = []
    minDays = 1000
    daysNeed = 15

    for item in stocks:
        code = item["code"]
        collection = client.Trade[code]
        res = list(collection.find({"close": {"$ne": 0}}, limit=daysNeed, sort=[("date", pymongo.DESCENDING)]))
        if len(res) < daysNeed:
            continue
        days = (today - res[-1]["date"]).days
        if days > minDays:
            continue
        if res[0]["close"] < res[1]["close"]:
            continue
        if days < minDays:
            minDays = days
            found = []
        p5 = 0
        p10 = 0
        p15 = 0
        for i in range(daysNeed):
            if i < 15:
                p15 = p15 + res[i]["close"]
            if i < 10:
                p10 = p10 + res[i]["close"]
            if i < 5:
                p5 = p5 + res[i]["close"]
        p5 = p5 / 5
        p10 = p10 / 10
        p15 = p15 / 15
        close = res[0]["close"]
        dp5 = p5 - p10
        dp10 = p10 - p15
        
        if dp5 >= 0.1 and dp10 >= 0.1 and dp5 > dp10: 
            item = {
                "code": code,
                "name": item["name"],
                "current": close,
                "ma5": p5,
                "ma10": p10,
                "ma15": p15,
                "factor": dp5 / dp10,
            }
            found.append(item)

    found = sorted(found, key=lambda item: item["factor"], reverse=True)[:10]
    for item in found:
        data = copy.copy(item)
        data["date"] = today
        data["strategy"] = 5
        SaveResult(client, data)
        
    client.close()
    
    if len(found) > 0:
        df = pandas.DataFrame(found,
                              columns=["code", "name", "factor", "current", "ma5",
                                       "ma10", "ma15"])
    else:
        return None
    
    # Write Excel
    sheet_name = u"策略5_趋势加速"
    wb, ws, path = OpenXlsx(sheet_name)

    ws.row_dimensions[1].height = 20
    for i, v in enumerate([14, 14, 14, 14, 14, 14]):
        ws.column_dimensions[openpyxl.utils.get_column_letter(i+1)].width = v

    WriteHead(df, ws)

    for i, row in df.iterrows():
        for j, v in enumerate(row.values):
            cell = ws.cell(row=i+2, column=j+1)
            cell.value = v

    wb.save(path)

    return df

In [7]:
def FilterSameStrategy():
    client = ConnectDB()
    today = Today()
    
    stocks = {}
    for s in client.Result.Recommend.find({"date": today}):
        code = s["code"]
        if code not in stocks:
            stocks[code] = {
                "code": code,
                "name": s["name"],
                "strategies": []
            }
        stocks[code]["strategies"].append(s["strategy"])
        
    result = []
    for k, v in stocks.iteritems():
        if len(v["strategies"]) > 1:
            result.append({
                "code": k,
                "name": v["name"],
                "strategies": v["strategies"]
            })
    df = pandas.DataFrame(result)
    
    # Write Excel
    sheet_name = u"汇总结果"
    wb, ws, path = OpenXlsx(sheet_name)

    ws.row_dimensions[1].height = 20
    for i, v in enumerate([14, 14, 18]):
        ws.column_dimensions[openpyxl.utils.get_column_letter(i+1)].width = v

    WriteHead(df, ws)

    for i, row in df.iterrows():
        for j, v in enumerate(row.values):
            cell = ws.cell(row=i+2, column=j+1)
            if j == 2:
                cell.value = json.dumps(v)
            else:
                cell.value = v

    wb.save(path)

    return df

In [8]:
def Callback():
    client = ConnectDB()
    today = Today()
    minDate = today + datetime.timedelta(days=-10)
    collection = client.Result.Recommend
    cursor = collection.find({"date": {"$gt": minDate}}) \
        .sort("date", pymongo.DESCENDING)

    stocks = {}
    daysDiff = -1
    daysCount = 0
    daysNeed = 4
    startDate = None
    for s in cursor:
        diff = (today - s["date"]).days
        if diff > daysDiff:
            daysDiff = diff
            daysCount = daysCount + 1
        if daysCount > daysNeed:
            break
        if daysCount < daysNeed:
            continue

        startDate = s["date"]
        code = s["code"]
        if code not in stocks:
            stocks[code] = {
                "code": code,
                "name": s["name"],
                "strategies": [s["strategy"]]
            }
        else:
            stocks[code]["strategies"].append(s["strategy"])

    found = []
    for code, stock in stocks.iteritems():
        res = []
        for s in client.Trade[code].find({"date": {"$gt": startDate}}) \
            .sort("date", pymongo.ASCENDING):
            res.append(s)
            
        if len(res) < daysNeed - 1:
            continue
        buyPrice = res[0]["open"]
        sellPrice1 = res[1]["high"]
        sellPrice2 = res[2]["high"]
        if buyPrice == 0:
            continue
        change1 = (sellPrice1 / buyPrice - 1) * 100
        change2 = (sellPrice2 / buyPrice - 1) * 100
        found.append({
            "code": code,
            "name": stock["name"],
            "strategies": stock["strategies"],
            "change1": change1,
            "change2": change2,
            "buy": buyPrice,
            "sell1": sellPrice1,
            "sell2": sellPrice2
        })

    found = sorted(found, key=lambda item: item["change2"], reverse=True)
    df = pandas.DataFrame(found, columns=["code", "name", "change1", "change2",
                                          "buy", "sell1", "sell2", "strategies"])
    
    
    # Write Excel
    sheet_name = u"回测结果"
    wb, ws, path = OpenXlsx(sheet_name)

    ws.row_dimensions[1].height = 20
    for i, v in enumerate([14, 14, 14, 14, 14, 14, 14, 18]):
        ws.column_dimensions[openpyxl.utils.get_column_letter(i+1)].width = v

    WriteHead(df, ws)

    for i, row in df.iterrows():
        for j, v in enumerate(row.values):
            cell = ws.cell(row=i+2, column=j+1)
            if j == 7:
                cell.value = json.dumps(v)
            else:
                cell.value = v

    wb.save(path)

    return df

In [13]:
print("Strategy 1")
Strategy_1()

print("Strategy 2")
Strategy_2()

print("Strategy 3")
Strategy_3()

print("Strategy 4")
Strategy_4()

print("Strategy 5")
Strategy_5()

print("Summary")
FilterSameStrategy()

print("Callback")
Callback2()

print "Done"

Strategy 1
Strategy 2
Strategy 3
Strategy 4
Strategy 5
Summary
Callback


,code,name,change1,change2,buy,sell1,sell2,strategies
0,000014,沙河股份,21.007194,23.093525,13.90,16.82,17.11,"[1, 3]"
1,002931,锋龙股份,11.093248,22.206592,49.76,55.28,60.81,[4]
2,300157,恒泰艾普,9.228442,17.095310,6.61,7.22,7.74,[2]
3,300611,美力科技,9.987966,14.380265,16.62,18.28,19.01,[1]
4,002903,宇环数控,4.422701,10.704501,51.10,53.36,56.57,[4]
5,300634,彩讯股份,9.042877,10.587279,49.21,53.66,54.42,[4]
6,002889,东方嘉盛,10.023795,9.726353,33.62,36.99,36.89,[4]
7,300515,三德科技,9.972678,9.221311,14.64,16.10,15.99,[1]
8,603301,振德医疗,11.696739,9.195948,63.18,70.57,68.99,[4]
9,600735,新华锦,7.415036,8.650875,9.71,10.43,10.55,[3]


In [41]:
def Callback_1(ws):
    client = ConnectDB()
    today = Today()
    minDate = today + datetime.timedelta(days=-20)
    collection = client.Result.Recommend
    cursor = collection.find({"date": {"$gt": minDate}}) \
        .sort("date", pymongo.DESCENDING)

    stocks = {}
    daysDiff = -1
    daysCount = 0
    daysNeed = 6
    startDate = None
    for s in cursor:
        diff = (today - s["date"]).days
        if diff > daysDiff:
            daysDiff = diff
            daysCount = daysCount + 1
        if daysCount > daysNeed:
            break
        if daysCount < daysNeed:
            continue

        startDate = s["date"]
        code = s["code"]
        if code not in stocks:
            stocks[code] = {
                "code": code,
                "name": s["name"],
                "strategies": [s["strategy"]]
            }
        else:
            stocks[code]["strategies"].append(s["strategy"])

    found = []
    for code, stock in stocks.iteritems():
        res = []
        for s in client.Trade[code].find({"date": {"$gt": startDate}}) \
            .sort("date", pymongo.ASCENDING):
            res.append(s)
            
        if len(res) < daysNeed - 1:
            continue
        
        buyPrice = res[0]["open"]
        if buyPrice == 0:
            continue

        maxResult = {
            "change": -100
        }
        for i, r in enumerate(res[1:]):
            sellPrice = r["high"]
            change = (sellPrice / buyPrice - 1) * 100
            if change > maxResult["change"]:
                maxResult = {
                    "change": change,
                    "date": i + 1,
                    "sell": r["high"]
                }
        if maxResult["change"] > -100:
            found.append({
                "recommend_date": startDate.date(),
                "code": code,
                "name": stock["name"],
                "strategies": stock["strategies"],
                "change": maxResult["change"],
                "buy": buyPrice,
                "sell": maxResult["sell"],
                "sell_date": maxResult["date"]
            })

    found = sorted(found, key=lambda item: item["change"], reverse=True)
    df = pandas.DataFrame(found, columns=["code", "name", "change",
                                          "buy", "sell", "sell_date", "strategies", "recommend_date"])
    
    

    ws.row_dimensions[1].height = 20
    for i, v in enumerate([14, 14, 14, 14, 14, 14, 18, 18]):
        ws.column_dimensions[openpyxl.utils.get_column_letter(i+1)].width = v

    WriteHead(df, ws)

    for i, row in df.iterrows():
        for j, v in enumerate(row.values):
            cell = ws.cell(row=i+2, column=j+1)
            if j == 6:
                cell.value = json.dumps(v)
            else:
                cell.value = v

    return df

def Callback_2(df, ws):
    count = {}
    total = 0
    for _, row in df.iterrows():
        change = round(row["change"], 2)
        for s in row["strategies"]:
            if s not in count:
                count[s] = []
            count[s].append(change)
            total = total + abs(change)
    
    scores = []
    for s, changes in count.iteritems():
        scores.append({
            "strategy": int(s),
            "score": sum(changes) / total
        })
    scores = sorted(scores, key=lambda item: item["strategy"]) 
    df = pandas.DataFrame(scores, columns=["strategy", "score"])

    StartColumn = 10
    for i, v in enumerate([14, 14]):
        ws.column_dimensions[openpyxl.utils.get_column_letter(i+StartColumn)].width = v

    WriteHead(df, ws, [1, StartColumn])

    for i, row in df.iterrows():
        for j, v in enumerate(row.values):
            cell = ws.cell(row=i+2, column=j+StartColumn)
            cell.value = v
    
def Callback2():
    sheet_name = u"回测结果"
    wb, ws, path = OpenXlsx(sheet_name)
    
    df = Callback_1(ws)
    Callback_2(df, ws)
    wb.save(path)


Callback2()
print "Done"

Done
